In [1]:
import pandas as pd
#Loads in the data file
data = pd.read_csv('optdigits_raining.csv', low_memory=False, header=None)

target = data[64]
inputArr = data.drop(64,1)


In [9]:
#this cell holds the training algorithm

import random
import math

def neuralNetworkTrain(inputNodes, hiddenNodes, outputNodes, learnRate, epochs, inputArray, targetArray):
    
    iToHweights = []
    hToOweights = []
    hiddenBiases = []
    outputBiases = []
    targetOutputs = range(outputNodes)
    
    for i in range(0,(inputNodes * hiddenNodes)):
        iToHweights.append(float(random.randint(0,10))/10)
        
    for i in range(0,(outputNodes * hiddenNodes)):
        hToOweights.append(float(random.randint(0,10))/10)
    
    for i in range(0,hiddenNodes):
        hiddenBiases.append(float(random.randint(0,10))/10)
        
    for i in range(0,outputNodes):
        outputBiases.append(float(random.randint(0,10))/10)
        
    #iToHweights = [.15,.2,.25,.3] #holds i to h weights
    #hToOweights = [.4,.45,.5,.55] #holds h to o weights
    #hiddenBiases = [.35,.35] #hidden node biases
    #outputBiases = [.6,.6] #output node biases
    #inputs = [.05,.1] #inputs
        
    #train through whole data set for number of epochs
    for i in range(0,epochs):
        for i in range(0,3822):
        
            for i in range(0,10):
                targetOutputs[i] = 0
            targetOutputs[targetArray.iloc[i]] = 1

            inputs = inputArray.iloc[i]

            #forwardsPass
            weightC = 0
            totalH = []
            H = []
            totalO = []
            O = []

            #totalH
            for i in range(0, hiddenNodes):
                total = 0
                for j in range(0, len(inputs)):
                    total = total + iToHweights[weightC] * inputs[j]
                    weightC += 1

                total = total + hiddenBiases[i]
                totalH.append(total)

            #H
            for i in range(0, len(totalH)):
                H.append(1/(1+math.exp(-totalH[i])))

            weightC = 0

            #totalO
            for i in range(0,outputNodes):
                total = 0
                for j in range(0, hiddenNodes):
                    total = total + hToOweights[weightC] * H[j]
                    weightC += 1

                total = total + outputBiases[i]
                totalO.append(total)

            #O
            for i in range(0, len(totalO)):
                O.append(1/(1+math.exp(-totalO[i])))

            #totalError
            totalError = 0
            for i in range(0, len(O)):
                totalError = totalError + (.5* (targetOutputs[i] - O[i])**2)

            #backwardsPass for O weights
            weightChangesHtoO = []
            outputC = 0

            nodeCounter = 0
            for i in range(0, len(hToOweights)):
                weightChangesHtoO.append(-(targetOutputs[outputC] - O[outputC])*(O[outputC]*(1-O[outputC]))*(H[nodeCounter]))
                nodeCounter += 1
                if((i+1)%hiddenNodes == 0):
                    nodeCounter = 0
                if((i+1)%hiddenNodes == 0):
                    outputC += 1

            weightChangesItoH = []
            hToOCount = 0
            timesThruInputCount = 0
            inputC = 0

            #calculate output bias changes
            outputBiasChanges = []
            for i in range(0, len(outputBiases)):
                outputBiasChanges.append(-(targetOutputs[i] - O[i])*(O[i]*(1-O[i])))

            weightChangesItoH = []
            hToOCount = 0
            timesThruInputCount = 0
            inputC = 0

            #backwardsPass for H weights
            #Etotal/H
            for i in range(0, len(iToHweights)):
                hToOCount = 0
                runningTotal = 0
                for j in range(0, outputNodes):
                    a = -(targetOutputs[j]-O[j])
                    b = hToOweights[hToOCount + timesThruInputCount]
                    c = (O[j]*(1-O[j]))
                    d = (a*b*c)
                    runningTotal = runningTotal + d
                    #print(a, ' ', c, ' ', b, ' ', d)
                    hToOCount += hiddenNodes    


                weightChangesItoH.append(runningTotal*(H[timesThruInputCount]*(1-H[timesThruInputCount]))*inputs[inputC])
                inputC += 1
                if((i+1)%inputNodes == 0):
                    inputC = 0
                if((i+1)%inputNodes == 0):
                    timesThruInputCount += 1

            #calculate hidden bias changes
            hiddenBiasChanges = []
            for i in range(0, len(hiddenBiases)):
                runningTotal = 0
                counter2 = 0
                for j in range(0, outputNodes):
                    a = -(targetOuputs[j]-O[j])
                    b = hToOweights[i + counter2]
                    c = (O[j]*(1-O[j]))
                    d = (a*b*c)
                    runningTotal = runningTotal + d
                    #print(a, ' ', c, ' ', b, ' ', d)
                    counter2 += hiddenNodes   
                hiddenBiasChanges.append(runningTotal*(H[i]*(1-H[i])))

            #updateWeights
            for i in range(0, len(hToOweights)):
                hToOweights[i] = hToOweights[i] - learnRate*weightChangesHtoO[i]

            for i in range(0, len(iToHweights)):
                iToHweights[i] = iToHweights[i] - learnRate*weightChangesItoH[i]

            for i in range(0, len(hiddenBiases)):
                hiddenBiases[i] = hiddenBiases[i] - learnRate*hiddenBiasChanges[i]

            for i in range(0, len(outputBiases)):
                outputBiases[i] = outputBiases[i] - learnRate*outputBiasChanges[i]

    returnList = [iToHweights, hToOweights, hiddenBiases, outputBiases]
    return returnList
            
#call the algorithm specifiying number of input, hidden, and output nodes, learning rate, and epochs  
#numHidden = input('Enter the number of hidden nodes: ')
#neuralNetwork(2,2,2,.5,1)

In [5]:
#get user inputs

numHidden = int(input('Enter the number of Hidden Nodes: '))
numOutput = 10
learningRate = int(input('Enter the Learning Rate: '))
epochs = int(input('Enter the number of Epochs: '))
numInput = len(inputArr.columns)

Enter the number of Hidden Nodes: 30
Enter the Learning Rate: 3
Enter the number of Epochs: 1


In [10]:
### run traning algorithm
returnList = neuralNetworkTrain(numInput,numHidden,numOutput,learningRate,epochs,inputArr, target)

TypeError: 'range' object does not support item assignment

In [8]:
#separate return data into variables 

iToHweights = returnList[0]
hToOweights = returnList[1]
hiddenBiases = returnList[2]
outputBiases = returnList[3]

NameError: name 'returnList' is not defined

In [9]:
#this cell holds the predict method

def neuralNetworkPredict(inputArray, iToHweights, hToOweights, hiddenBiases, outputBiases, inputNodes, hiddenNodes, outputNodes, learnRate):
    weightC = 0
    totalH = []
    H = []
    totalO = []
    O = []
    
    #totalH
    for i in range(0, hiddenNodes):
        total = 0
        for j in range(0, len(inputArray)):
            total = total + iToHweights[weightC] * inputArray[j]
            weightC += 1

        total = total + hiddenBiases[i]
        totalH.append(total)
        
    #H
    for i in range(0, len(totalH)):
        H.append(1/(1+math.exp(-totalH[i])))
    
    weightC = 0

    #totalO
    for i in range(0,outputNodes):
        total = 0
        for j in range(0, hiddenNodes):
            total = total + hToOweights[weightC] * H[j]
            weightC += 1

        total = total + outputBiases[i]
        totalO.append(total)

    #O
    for i in range(0, len(totalO)):
        O.append(1/(1+math.exp(-totalO[i])))
        
    return O

In [10]:
testingData = pd.read_csv('optdigits_test.csv', low_memory=False, header=None)

In [23]:
#collect all of output data into testingOutput[]

testingOutput = []

for i in range (0, 1000):
    observation = testingInputArr.iloc[i]
    testingOutput.append(neuralNetworkPredict(observation, iToHweights, hToOweights, hiddenBiases, outputBiases, numInput, numHidden, numOutput, learningRate))

In [24]:
print(testingOutput)

[[0.9999998394232168, 0.9999999746638784, 0.9999945938055133, 0.9999999140860334, 0.9999999140860334, 0.9999998698771028, 0.9999998214635154, 0.9999986352518566, 0.9999999245654224, 0.9999885121979759], [0.9999998394232168, 0.9999999746638784, 0.9999945938055133, 0.9999999140860334, 0.9999999140860334, 0.9999998698771028, 0.9999998214635154, 0.9999986352518566, 0.9999999245654224, 0.9999885121979759], [0.9999998394232168, 0.9999999746638784, 0.9999945938055133, 0.9999999140860334, 0.9999999140860334, 0.9999998698771028, 0.9999998214635154, 0.9999986352518566, 0.9999999245654224, 0.9999885121979759], [0.9999998394232168, 0.9999999746638784, 0.9999945938055133, 0.9999999140860334, 0.9999999140860334, 0.9999998698771028, 0.9999998214635154, 0.9999986352518566, 0.9999999245654224, 0.9999885121979759], [0.9999998394232168, 0.9999999746638784, 0.9999945938055133, 0.9999999140860334, 0.9999999140860334, 0.9999998698771028, 0.9999998214635154, 0.9999986352518566, 0.9999999245654224, 0.99998851